#### SQL 50 (Leetcode)

###Recyclable and Low Fat Products
####Q1. Write a solution to find the ids of products that are both low fat and recyclable.

####Input: 
Products table:

| product_id  | low_fats | recyclable |
|-------------|----------|------------|
| 0           | Y        | N          |
| 1           | Y        | Y          |
| 2           | N        | Y          |
| 3           | Y        | Y          |
| 4           | N        | N          |

####Output: 
| product_id  |
|-------------|
| 1           |
| 3           |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# dataframe
data = [(0, "Y", "N"),
        (1, "Y", "Y"),
        (2, "N", "Y"),
        (3, "Y", "Y")]
schema = "product_id int, low_fats string, recyclable string"
df = spark.createDataFrame(data=data, schema=schema)

# Solution
result_df = df.where((col("low_fats") == "Y") & (col("recyclable") == "Y")).select(col("product_id"))
display(result_df)

product_id
1
3


###Find Customer Referee
####Q2. Find the names of the customer that are not referred by the customer with id = 2.

Return the result table in any order.

The result format is in the following example.

####Input: 
Customer table:

| id | name | referee_id |
|----|------|------------|
| 1  | Will | null       |
| 2  | Jane | null       |
| 3  | Alex | 2          |
| 4  | Bill | null       |
| 5  | Zack | 1          |
| 6  | Mark | 2          |

####Output: 
| name |
|------|
| Will |
| Jane |
| Bill |
| Zack |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, lit, coalesce

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# dataframe
data = [
    (1, "Will", None),
    (2, "Jane", None),
    (3, "Alex", 2),
    (4, "Bill", None),
    (5, "Zack", 1),
    (6, "Mark", 2)
]
schema = StructType([StructField("id", IntegerType(), False),
                    StructField("name", StringType(), False),
                    StructField("referee_id", IntegerType(), True)])
df = spark.createDataFrame(data=data, schema=schema)

# solution
result_df = df.withColumn("referee_id", coalesce(col("referee_id"), lit(-1))) \
            .where(col("referee_id") != 2).select(col("name")).distinct()
display(result_df)

name
Will
Jane
Bill
Zack


###Big Countries

A country is big if:

- it has an area of at least three million (i.e., 3000000 km2), or

- it has a population of at least twenty-five million (i.e., 25000000).

####Q3. Write a solution to find the name, population, and area of the big countries. Return the result table in any order.

The result format is in the following example.

####Input: 
World table:

| name        | continent | area    | population | gdp          |
|-------------|-----------|---------|------------|--------------|
| Afghanistan | Asia      | 652230  | 25500100   | 20343000000  |
| Albania     | Europe    | 28748   | 2831741    | 12960000000  |
| Algeria     | Africa    | 2381741 | 37100000   | 188681000000 |
| Andorra     | Europe    | 468     | 78115      | 3712000000   |
| Angola      | Africa    | 1246700 | 20609294   | 100990000000 |

####Output: 
| name        | population | area    |
|-------------|------------|---------|
| Afghanistan | 25500100   | 652230  |
| Algeria     | 37100000   | 2381741 |

In [0]:
# imports 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# spark session
spark = SparkSession.builder.appName("sparkApp").getOrCreate()

# dataframe
data = [
        ("Afghanistan",	"Asia",	652230,	25500100, 20343000000),
        ("Albania", "Europe", 28748, 2831741, 12960000000),
        ("Algeria", "Africa", 2381741, 37100000, 188681000000),
        ("Andorra", "Europe", 468, 78115, 3712000000),
        ("Angola", "Africa", 1246700, 20609294, 100990000000)
]
schema = "name string, continent string, area long, population long, gdp long"
df = spark.createDataFrame(data=data, schema=schema)

# solution
result_df = df.where((col("area") >= 3000000) | (col("population") >= 25000000)) \
            .select(col("name"), col("population"), col("area")).distinct()
display(result_df)

name,population,area
Afghanistan,25500100,652230
Algeria,37100000,2381741


###Article Views I

####Q.4 Write a solution to find all the authors that viewed at least one of their own articles. Return the result table sorted by id in ascending order.

The result format is in the following example.

####Input: 
Views table:

| article_id | author_id | viewer_id | view_date  |
|------------|-----------|-----------|------------|
| 1          | 3         | 5         | 2019-08-01 |
| 1          | 3         | 6         | 2019-08-02 |
| 2          | 7         | 7         | 2019-08-01 |
| 2          | 7         | 6         | 2019-08-02 |
| 4          | 7         | 1         | 2019-07-22 |
| 3          | 4         | 4         | 2019-07-21 |
| 3          | 4         | 4         | 2019-07-21 |

####Output: 
| id   |
|------|
| 4    |
| 7    |

In [0]:
# imports 
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# dataframe
data = [
    (1, 3, 5, '2019-08-01'),
    (1, 3, 6, '2019-08-02'),
    (2, 7, 7, '2019-08-01'),
    (2, 7, 6, '2019-08-02'),
    (4, 7, 1, '2019-07-22'),
    (3, 4, 4, '2019-07-21'),
    (3, 4, 4, '2019-07-21')
]
schema = "article_id int, author_id int, viewer_id int, view_date string"
df = spark.createDataFrame(data=data, schema=schema)
df = df.withColumn("view_date", to_date(col("view_date"), 'yyyy-MM-dd'))

# solution
result_df = df.filter(col("author_id") == col("viewer_id")) \
    .select(col("author_id").alias("id")).distinct().orderBy("id")
display(result_df)

id
4
7


###Invalid Tweets

####Q.5 Write a solution to find the IDs of the invalid tweets. The tweet is invalid if the number of characters used in the content of the tweet is strictly greater than 15.

Return the result table in any order.

The result format is in the following example.

####Input: 
Tweets table:

| tweet_id | content                          |
|----------|----------------------------------|
| 1        | Vote for Biden                   |
| 2        | Let us make America great again! |

####Output: 
| tweet_id |
|----------|
| 2        |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [(1, 'Vote for Biden'),
        (2, 'Let us make America great again!')]
schema = "tweet_id integer, content string"
df = spark.createDataFrame(data=data, schema=schema)

# solution
result_df = df.filter(length(col("content")) > 15).select(col("tweet_id"))
display(result_df)

tweet_id
2


###Replace Employee ID With The Unique Identifier
####Q.6 Write a solution to show the unique ID of each user, If a user does not have a unique ID replace just show null.Return the result table in any order.

The result format is in the following example.

####Input: 
Employees table:

| id | name     |
|----|----------|
| 1  | Alice    |
| 7  | Bob      |
| 11 | Meir     |
| 90 | Winston  |
| 3  | Jonathan |

EmployeeUNI table:

| id | unique_id |
|----|-----------|
| 3  | 1         |
| 11 | 2         |
| 90 | 3         |

####Output: 
| unique_id | name     |
|-----------|----------|
| null      | Alice    |
| null      | Bob      |
| 2         | Meir     |
| 3         | Winston  |
| 1         | Jonathan |

In [0]:
# imports 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce

# spark sesion
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data1 = [(1, "Alice"),
         (7, "Bob"),
         (11, "Meir"),
         (90, "Winston"),
         (3, "Jonathan")]
schema1 = "id integer, name string"
df1 = spark.createDataFrame(data=data1, schema=schema1)
data2 = [(3, 1),
         (11, 2),
         (90, 3)]
schema2 = "id integer, unique_id integer"
df2 = spark.createDataFrame(data=data2, schema=schema2)

# solution
joined_df = df1.join(df2, df1.id == df2.id, "left_outer")
result_df = joined_df.select(col("unique_id"), col("name"))
display(result_df)

unique_id,name
null,Alice
null,Bob
2,Meir
3,Winston
1,Jonathan


###Product Sales Analysis I
####Q.7 Write a solution to report the product_name, year, and price for each sale_id in the Sales table. Return the resulting table in any order.

The result format is in the following example.

####Input: 
Sales table:

| sale_id | product_id | year | quantity | price |
|---------|------------|------|----------|-------|
| 1       | 100        | 2008 | 10       | 5000  |
| 2       | 100        | 2009 | 12       | 5000  |
| 7       | 200        | 2011 | 15       | 9000  |

Product table:

| product_id | product_name |
|------------|--------------|
| 100        | Nokia        |
| 200        | Apple        |
| 300        | Samsung      |

####Output: 
| product_name | year  | price |
|--------------|-------|-------|
| Nokia        | 2008  | 5000  |
| Nokia        | 2009  | 5000  |
| Apple        | 2011  | 9000  |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe1
data1 =  [(1, 100, 2008, 10, 5000),
          (2, 100, 2009, 12, 5000),
          (7, 200, 2011, 15, 9000)]

schema1 = "sale_id int, product_id int, year int, quantity int, price int"

df_sales = spark.createDataFrame(data=data1, schema=schema1)

# input dataframe2
data2 =  [(100, "Nokia"),
          (200, "Apple"),
          (300, "Samsung")]

schema2 = "product_id int, product_name string"

df_product = spark.createDataFrame(data=data2, schema=schema2)

# solution
joined_df = df_sales.join(df_product, df_sales.product_id == df_product.product_id, "inner")
result_df = joined_df.select(col("product_name"), col("year"), col("price"))
display(result_df)

product_name,year,price
Nokia,2008,5000
Nokia,2009,5000
Apple,2011,9000


###Customer Who Visited but Did Not Make Any Transactions
####Q.8 Write a solution to find the IDs of the users who visited without making any transactions and the number of times they made these types of visits. Return the result table sorted in any order.

The result format is in the following example.

####Input:
Visits

| visit_id | customer_id |
|----------|-------------|
| 1        | 23          |
| 2        | 9           |
| 4        | 30          |
| 5        | 54          |
| 6        | 96          |
| 7        | 54          |
| 8        | 54          |

Transactions

| transaction_id | visit_id | amount |
|----------------|----------|--------|
| 2              | 5        | 310    |
| 3              | 5        | 300    |
| 9              | 5        | 200    |
| 12             | 1        | 910    |
| 13             | 2        | 970    |

####Output: 
| customer_id | count_no_trans |
|-------------|----------------|
| 54          | 2              |
| 30          | 1              |
| 96          | 1              |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data1 = [(1, 23),
         (2, 9),
         (4, 30),
         (5, 54),
         (6, 96),
         (7, 54),
         (8, 54)]
schema1 = "visit_id int, customer_id int"
Visits_df = spark.createDataFrame(data=data1, schema=schema1)
data2 = [(2, 5, 310),
         (3, 5, 300),   
         (9, 5, 200),
         (12, 1, 910),
         (13, 2, 970)]
schema2 = "transaction_id int, visit_id int, amount int"
transactions_df = spark.createDataFrame(data=data2, schema=schema2)

# solution
joined_df = Visits_df.join(transactions_df, Visits_df.visit_id == transactions_df.visit_id, "left_outer")
result_df = joined_df.filter(col("transaction_id").isNull()).groupBy(col("customer_id")) \
            .agg(count("*").alias("count_no_trans")).sort(col("count_no_trans").desc())
display(result_df)

customer_id,count_no_trans
54,2
96,1
30,1


###Rising Temperature
####Q.9 Write a solution to find all dates' Id with higher temperatures compared to its previous dates (yesterday). Return the result table in any order.

The result format is in the following example.

####Input: 
Weather table:

| id | recordDate | temperature |
|----|------------|-------------|
| 1  | 2015-01-01 | 10          |
| 2  | 2015-01-02 | 25          |
| 3  | 2015-01-03 | 20          |
| 4  | 2015-01-04 | 30          |

####Output: 
| id |
|----|
| 2  |
| 4  |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, lag
from pyspark.sql.window import Window

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [(1, '2015-01-01', 10),
        (2, '2015-01-02', 25),        
        (3, '2015-01-03', 20),          
        (4, '2015-01-04', 30)]
schema = ["id", "recordDate", "temperature"]
df = spark.createDataFrame(data=data, schema=schema)
df = df.withColumn("recordDate", to_date(col("recordDate"), "yyyy-MM-dd"))

# solution
window_spec = Window.orderBy("id")
result_df = df.withColumn("previous_temperature", lag("temperature", 1).over(window_spec)) \
            .filter(col("temperature") > col("previous_temperature")).select(col("id"))
display(result_df)

id
2
4


###Average Time of Process per Machine
####Q.10 There is a factory website that has several machines each running the same number of processes. Write a solution to find the average time each machine takes to complete a process. The time to complete a process is the 'end' timestamp minus the 'start' timestamp. The average time is calculated by the total time to complete every process on the machine divided by the number of processes that were run. The resulting table should have the machine_id along with the average time as processing_time, which should be rounded to 3 decimal places.

Return the result table in any order.

The result format is in the following example.

####Input: 
Activity table:

| machine_id | process_id | activity_type | timestamp |
|------------|------------|---------------|-----------|
| 0          | 0          | start         | 0.712     |
| 0          | 0          | end           | 1.520     |
| 0          | 1          | start         | 3.140     |
| 0          | 1          | end           | 4.120     |
| 1          | 0          | start         | 0.550     |
| 1          | 0          | end           | 1.550     |
| 1          | 1          | start         | 0.430     |
| 1          | 1          | end           | 1.420     |
| 2          | 0          | start         | 4.100     |
| 2          | 0          | end           | 4.512     |
| 2          | 1          | start         | 2.500     |
| 2          | 1          | end           | 5.000     |

####Output: 
| machine_id | processing_time |
|------------|-----------------|
| 0          | 0.894           |
| 1          | 0.995           |
| 2          | 1.456           |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lead, avg, round
from pyspark.sql.window import Window

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (0, 0, 'start', 0.712),
    (0, 0, 'end', 1.520),
    (0, 1, 'start', 3.140),
    (0, 1, 'end', 4.120),
    (1, 0, 'start', 0.550),
    (1, 0, 'end', 1.550),
    (1, 1, 'start', 0.430),
    (1, 1, 'end', 1.420),
    (2, 0, 'start', 4.100),
    (2, 0, 'end', 4.512),
    (2, 1, 'start', 2.500),
    (2, 1, 'end', 5.000)
]
schema = "machine_id int, process_id int, activity_type string, timestamp float"
df = spark.createDataFrame(data=data, schema=schema)

#solution
window_spec = Window.partitionBy(col("machine_id"), col("process_id")).orderBy(col("timestamp"))
lead_df = df.withColumn("lead_timestamp", lead(col("timestamp"), 1).over(window_spec)) \
         .filter(col("activity_type") == 'start')
diff_df = lead_df.withColumn("timestamp_diff", col("lead_timestamp")-col("timestamp"))
avg_df = diff_df.groupBy(col("machine_id")).agg(avg(col("timestamp_diff")).alias("processing_time"))
result_df = avg_df.withColumn("processing_time", round(col("processing_time"), 3)).sort(col("processing_time"))
display(result_df)

machine_id,processing_time
0,0.894
1,0.995
2,1.456


###Employee Bonus
####Q.11 Write a solution to report the name and bonus amount of each employee with a bonus less than 1000.

Return the result table in any order.

The result format is in the following example.

####Input: 
Employee table:

| empId | name   | supervisor | salary |
|-------|--------|------------|--------|
| 3     | Brad   | null       | 4000   |
| 1     | John   | 3          | 1000   |
| 2     | Dan    | 3          | 2000   |
| 4     | Thomas | 3          | 4000   |

Bonus table:

| empId | bonus |
|-------|-------|
| 2     | 500   |
| 4     | 2000  |

####Output: 
| name | bonus |
|------|-------|
| Brad | null  |
| John | null  |
| Dan  | 500   |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data1 = [
    (3, 'Brad', None, 4000),
    (1, 'John', 3, 1000),
    (2, 'Dan', 3, 2000),
    (4, 'Thomas', 3, 4000)
]
schema1 = "empId int, name string, supervisor int, salary int"
employee_df = spark.createDataFrame(data=data1, schema=schema1)
data2 = [
    (2, 500),
    (4, 2000)
]
schema2 = "empId int, bonus int"
bonus_df = spark.createDataFrame(data=data2, schema=schema2)

# solution
joined_df = employee_df.join(bonus_df, employee_df.empId == bonus_df.empId, "left_outer")
result_df = joined_df.filter((col("bonus") < 1000) | (col("bonus").isNull())).select(col("name"), col("bonus"))
display(result_df)

name,bonus
Brad,null
John,null
Dan,500


###Students and Examinations
####Q.12 Write a solution to find the number of times each student attended each exam.

Return the result table ordered by student_id and subject_name.

The result format is in the following example.

####Input: 
Students table:

| student_id | student_name |
|------------|--------------|
| 1          | Alice        |
| 2          | Bob          |
| 13         | John         |
| 6          | Alex         |

Subjects table:

| subject_name |
|--------------|
| Math         |
| Physics      |
| Programming  |

Examinations table:

| student_id | subject_name |
|------------|--------------|
| 1          | Math         |
| 1          | Physics      |
| 1          | Programming  |
| 2          | Programming  |
| 1          | Physics      |
| 1          | Math         |
| 13         | Math         |
| 13         | Programming  |
| 13         | Physics      |
| 2          | Math         |
| 1          | Math         |

####Output: 
| student_id | student_name | subject_name | attended_exams |
|------------|--------------|--------------|----------------|
| 1          | Alice        | Math         | 3              |
| 1          | Alice        | Physics      | 2              |
| 1          | Alice        | Programming  | 1              |
| 2          | Bob          | Math         | 1              |
| 2          | Bob          | Physics      | 0              |
| 2          | Bob          | Programming  | 1              |
| 6          | Alex         | Math         | 0              |
| 6          | Alex         | Physics      | 0              |
| 6          | Alex         | Programming  | 0              |
| 13         | John         | Math         | 1              |
| 13         | John         | Physics      | 1              |
| 13         | John         | Programming  | 1              |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, lit

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataFrame
data1 = [
    (1, "Alice"),
    (2, "Bob"),
    (13, "John"),
    (6, "Alex")
]
schema1 = "student_id int, student_name string"
student_df = spark.createDataFrame(data=data1, schema=schema1)
data2 = [
    ("Math",),
    ("Physics",),
    ("Programming",)
]
schema2 = "subject_name string"
subject_df = spark.createDataFrame(data=data2, schema=schema2)
data3 = [
    (1, "Math"),
    (1, "Physics"),
    (1, "Programming"),
    (2, "Programming"),
    (1, "Physics"),
    (1, "Math"),
    (13, "Math"),
    (13, "Programming"),
    (13, "Physics"),
    (2, "Math"),
    (1, "Math")
]
schema3 = "student_id int, subject_name string"
examination_df = spark.createDataFrame(data=data3, schema=schema3)

# solution
student_subject_df = student_df.crossJoin(subject_df)
count_df = examination_df.groupBy(col("student_id"), col("subject_name")).agg(count("*").alias("attended_exams"))
result_df = student_subject_df.join(count_df, (student_subject_df.student_id == count_df.student_id) & \
            (student_subject_df.subject_name == count_df.subject_name), "left_outer") \
            .select(student_subject_df.student_id, col("student_name"), student_subject_df.subject_name, \
             col("attended_exams")).fillna(0, subset=["attended_exams"])
result_df = result_df.orderBy(col("student_id"), col("subject_name"))
display(result_df)

student_id,student_name,subject_name,attended_exams
1,Alice,Math,3
1,Alice,Physics,2
1,Alice,Programming,1
2,Bob,Math,1
2,Bob,Physics,0
2,Bob,Programming,1
6,Alex,Math,0
6,Alex,Physics,0
6,Alex,Programming,0
13,John,Math,1


###Managers with at Least 5 Direct Reports
####Q.13 Write a solution to find managers with at least five direct reports.

Return the result table in any order.

The result format is in the following example.

####Input: 
Employee table:

| id  | name  | department | managerId |
|-----|-------|------------|-----------|
| 101 | John  | A          | null      |
| 102 | Dan   | A          | 101       |
| 103 | James | A          | 101       |
| 104 | Amy   | A          | 101       |
| 105 | Anne  | A          | 101       |
| 106 | Ron   | B          | 101       |

####Output: 
| name |
|------|
| John |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (101, "John", "A", None),
    (102, "Dan", "A", 101),
    (103, "James", "A", 101),
    (104, "Amy", "A", 101),
    (105, "Anne", "A", 101),
    (106, "Ron", "B", 101)
]
schema = "id int, name string, department string, managerId int"
df = spark.createDataFrame(data=data, schema=schema)
df1 = df.alias("df1")
df2 = df.alias("df2")

# solution
count_df = df1.groupBy(col("df1.managerId")).agg(count("*").alias("reporting_count")) \
          .filter(col("reporting_count") >= 5)
result_df = count_df.join(df2, col("df1.managerId") == col("df2.id"), "inner").select(col("df2.name"))
display(result_df)

name
John


###Confirmation Rate
####Q.14 The confirmation rate of a user is the number of 'confirmed' messages divided by the total number of requested confirmation messages. The confirmation rate of a user that did not request any confirmation messages is 0. Round the confirmation rate to two decimal places. Write a solution to find the confirmation rate of each user.

Return the result table in any order.

The result format is in the following example.

####Input: 
Signups table:

| user_id | time_stamp          |
|---------|---------------------|
| 3       | 2020-03-21 10:16:13 |
| 7       | 2020-01-04 13:57:59 |
| 2       | 2020-07-29 23:09:44 |
| 6       | 2020-12-09 10:39:37 |

Confirmations table:

| user_id | time_stamp          | action    |
|---------|---------------------|-----------|
| 3       | 2021-01-06 03:30:46 | timeout   |
| 3       | 2021-07-14 14:00:00 | timeout   |
| 7       | 2021-06-12 11:57:29 | confirmed |
| 7       | 2021-06-13 12:58:28 | confirmed |
| 7       | 2021-06-14 13:59:27 | confirmed |
| 2       | 2021-01-22 00:00:00 | confirmed |
| 2       | 2021-02-28 23:59:59 | timeout   |

####Output: 
| user_id | confirmation_rate |
|---------|-------------------|
| 6       | 0.00              |
| 3       | 0.00              |
| 7       | 1.00              |
| 2       | 0.50              |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, round

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data1 = [
    (3, "2020-03-21 10:16:13"),
    (7, "2020-01-04 13:57:59"),
    (2, "2020-07-29 23:09:44"),
    (6, "2020-12-09 10:39:37")
]
schema1 = "user_id int, time_stamp string"
signups_df = spark.createDataFrame(data=data1, schema=schema1)
data2 = [
    (3, "2021-01-06 03:30:46", "timeout"),
    (3, "2021-07-14 14:00:00", "timeout"),
    (7, "2021-06-12 11:57:29", "confirmed"),
    (7, "2021-06-13 12:58:28", "confirmed"),
    (7, "2021-06-14 13:59:27", "confirmed"),
    (2, "2021-01-22 00:00:00", "confirmed"),
    (2, "2021-02-28 23:59:59", "timeout")
]
schema2 = "user_id int, time_stamp string, action string"
confirmations_df = spark.createDataFrame(data=data2, schema=schema2)

# solution
aggregated_df = confirmations_df.groupBy("user_id") \
    .agg(
        count("*").alias("total_actions"),
        count(when(col("action") == "confirmed", True)).alias("confirmed_actions")
    )
result_df = aggregated_df.withColumn(
    "confirmation_rate", 
    round(col("confirmed_actions") / col("total_actions"), 2)
)
final_df = signups_df.join(result_df, "user_id", "left").select(
    col("user_id"),
    col("confirmation_rate")
).fillna({"confirmation_rate": 0.00}).sort(col("confirmation_rate"))
display(final_df)

user_id,confirmation_rate
3,0.0
6,0.0
2,0.5
7,1.0


###Not Boring Movies
####Q.15 Write a solution to report the movies with an odd-numbered ID and a description that is not "boring".

Return the result table ordered by rating in descending order.

The result format is in the following example.

####Input: 
Cinema table:

| id | movie      | description | rating |
|----|------------|-------------|--------|
| 1  | War        | great 3D    | 8.9    |
| 2  | Science    | fiction     | 8.5    |
| 3  | irish      | boring      | 6.2    |
| 4  | Ice song   | Fantacy     | 8.6    |
| 5  | House card | Interesting | 9.1    |

####Output: 
| id | movie      | description | rating |
|----|------------|-------------|--------|
| 5  | House card | Interesting | 9.1    |
| 1  | War        | great 3D    | 8.9    |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (1, "War", "great 3D", 8.9),
    (2, "Science", "fiction", 8.5),
    (3, "irish", "boring", 6.2),
    (4, "Ice song", "Fantasy", 8.6),
    (5, "House card", "Interesting", 9.1)
]
schema = "id int, movie string, description string, rating float"
df = spark.createDataFrame(data=data, schema=schema)

# solution
result_df = df.filter((col("id")%2!=0) & (col("description")!="boring")).sort(col("rating").desc()).select("*")
display(result_df)

id,movie,description,rating
5,House card,Interesting,9.1
1,War,great 3D,8.9


###Average Selling Price
####Q.16 Write a solution to find the average selling price for each product. average_price should be rounded to 2 decimal places.

Return the result table in any order.

The result format is in the following example.

####Input: 
Prices table:

| product_id | start_date | end_date   | price  |
|------------|------------|------------|--------|
| 1          | 2019-02-17 | 2019-02-28 | 5      |
| 1          | 2019-03-01 | 2019-03-22 | 20     |
| 2          | 2019-02-01 | 2019-02-20 | 15     |
| 2          | 2019-02-21 | 2019-03-31 | 30     |

UnitsSold table:

| product_id | purchase_date | units |
|------------|---------------|-------|
| 1          | 2019-02-25    | 100   |
| 1          | 2019-03-01    | 15    |
| 2          | 2019-02-10    | 200   |
| 2          | 2019-03-22    | 30    |

####Output: 
| product_id | average_price |
|------------|---------------|
| 1          | 6.96          |
| 2          | 16.96         |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, round

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data1 = [
    (1, "2019-02-17", "2019-02-28", 5),
    (1, "2019-03-01", "2019-03-22", 20),
    (2, "2019-02-01", "2019-02-20", 15),
    (2, "2019-02-21", "2019-03-31", 30)
]
schema1 = "product_id int, start_date string, end_date string, price int"
prices_df = spark.createDataFrame(data=data1, schema=schema1)
data2 = [
    (1, "2019-02-25", 100),
    (1, "2019-03-01", 15),
    (2, "2019-02-10", 200),
    (2, "2019-03-22", 30)
]
schema2 = "product_id int, purchase_date string, units int"
unitsold_df = spark.createDataFrame(data=data2, schema=schema2)

# Solution
joined_df = prices_df.join(unitsold_df, 
                           (prices_df.product_id == unitsold_df.product_id) & 
                           (unitsold_df.purchase_date >= prices_df.start_date) & 
                           (unitsold_df.purchase_date <= prices_df.end_date),
                           "inner") \
            .select(prices_df.product_id, col("price"), col("units"))
result_df = joined_df.groupBy(col("product_id")) \
    .agg(sum(col("price") * col("units")).alias("total_revenue"),
         sum(col("units")).alias("total_units")) \
    .select(col("product_id"), 
            round(col("total_revenue") / col("total_units"), 2).alias("average_price"))
display(result_df)

product_id,average_price
1,6.96
2,16.96


###Project Employees I
####Q.17 Write an SQL query that reports the average experience years of all the employees for each project, rounded to 2 digits.

Return the result table in any order.

The query result format is in the following example.

####Input: 
Project table:

| project_id  | employee_id |
|-------------|-------------|
| 1           | 1           |
| 1           | 2           |
| 1           | 3           |
| 2           | 1           |
| 2           | 4           |

Employee table:

| employee_id | name   | experience_years |
-------------|--------|------------------|
| 1           | Khaled | 3                |
| 2           | Ali    | 2                |
| 3           | John   | 1                |
| 4           | Doe    | 2                |

####Output: 
| project_id  | average_years |
|-------------|---------------|
| 1           | 2.00          |
| 2           | 2.50          |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, format_number

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data1 = [
    (1, 1),
    (1, 2),
    (1, 3),
    (2, 1),
    (2, 4)
]
schema1 = "project_id int, employee_id int"
project_df = spark.createDataFrame(data=data1, schema=schema1)
data2 = [
    (1, "Khaled", 3),
    (2, "Ali", 2),
    (3, "John", 1),
    (4, "Doe", 2)
]
schema2 = "employee_id int, name string, experience_years int"
employee_df = spark.createDataFrame(data=data2, schema=schema2)

# solution
joined_df = project_df.join(employee_df, project_df.employee_id == employee_df.employee_id, "inner") \
            .select(col("project_id"), project_df.employee_id.alias("employee_id"), col("experience_years"))
result_df = joined_df.groupBy(col("project_id")) \
            .agg(
                count("employee_id").alias("emp_count"),
                sum("experience_years").alias("total_experience")
            ) \
            .select(
                col("project_id"),
                format_number(col("total_experience") / col("emp_count"), 2).alias("average_years")
            )
display(result_df)

project_id,average_years
1,2.00
2,2.50


###Percentage of Users Attended a Contest
####Q.18 Write a solution to find the percentage of the users registered in each contest rounded to two decimals.

Return the result table ordered by percentage in descending order. In case of a tie, order it by contest_id in ascending order.

The result format is in the following example.

####Input: 
Users table:

| user_id | user_name |
|---------|-----------|
| 6       | Alice     |
| 2       | Bob       |
| 7       | Alex      |

Register table:

| contest_id | user_id |
|------------|---------|
| 215        | 6       |
| 209        | 2       |
| 208        | 2       |
| 210        | 6       |
| 208        | 6       |
| 209        | 7       |
| 209        | 6       |
| 215        | 7       |
| 208        | 7       |
| 210        | 2       |
| 207        | 2       |
| 210        | 7       |

####Output: 
| contest_id | percentage |
|------------|------------|
| 208        | 100.0      |
| 209        | 100.0      |
| 210        | 100.0      |
| 215        | 66.67      |
| 207        | 33.33      |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, asc, round

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data1 = [
    (6, "Alice"),
    (2, "Bob"),
    (7, "Alex")
]
schema1 = "user_id int, user_name string"
users_df = spark.createDataFrame(data=data1, schema=schema1)
data2 = [
    (215, 6),
    (209, 2),
    (208, 2),
    (210, 6),
    (208, 6),
    (209, 7),
    (209, 6),
    (215, 7),
    (208, 7),
    (210, 2),
    (207, 2),
    (210, 7)
]
schema2 = "contest_id int, user_id int"
register_df = spark.createDataFrame(data=data2, schema=schema2)

# solution
total_user_count = users_df.count()
result_df = register_df.groupBy(col("contest_id")) \
            .agg(count(col("user_id")).alias("participated_user_count")) \
            .select(col("contest_id"), 
                    round((col("participated_user_count")/total_user_count)*100, 2).alias("percentage")) \
            .orderBy(col("percentage").desc(), col("contest_id").asc())
display(result_df)

contest_id,percentage
208,100.0
209,100.0
210,100.0
215,66.67
207,33.33


###Queries Quality and Percentage
We define query quality as: 
The average of the ratio between query rating and its position.
We also define poor query percentage as:
The percentage of all queries with rating less than 3.

####Q.19 Write a solution to find each query_name, the quality and poor_query_percentage.

Both quality and poor_query_percentage should be rounded to 2 decimal places.

Return the result table in any order.

The result format is in the following example.

####Input: 
Queries table:

| query_name | result            | position | rating |
|------------|-------------------|----------|--------|
| Dog        | Golden Retriever  | 1        | 5      |
| Dog        | German Shepherd   | 2        | 5      |
| Dog        | Mule              | 200      | 1      |
| Cat        | Shirazi           | 5        | 2      |
| Cat        | Siamese           | 3        | 3      |
| Cat        | Sphynx            | 7        | 4      |

####Output: 
| query_name | quality | poor_query_percentage |
|------------|---------|-----------------------|
| Dog        | 2.50    | 33.33                 |
| Cat        | 0.66    | 33.33                 |

In [0]:
# imports 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, round, when

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    ("Dog", "Golden Retriever", 1, 5),
    ("Dog", "German Shepherd", 2, 5),
    ("Dog", "Mule", 200, 1),
    ("Cat", "Shirazi", 5, 2),
    ("Cat", "Siamese", 3, 3),
    ("Cat", "Sphynx", 7, 4)
]
schema = "query_name string, result string, position int, rating int"
df = spark.createDataFrame(data=data, schema=schema)

# solution
result_df = df.groupBy(col("query_name")) \
    .agg(
        round(sum(col("position") / col("rating")), 2).alias("quality"),
        round(sum(when(col("rating") < 3, 1).otherwise(0)) / count("*"), 2).alias("poor_query_percentage")
    ) \
    .select(col("query_name"), col("quality"), col("poor_query_percentage"))
display(result_df)

query_name,quality,poor_query_percentage
Dog,200.6,0.33
Cat,5.25,0.33


###Monthly Transactions I
####Q.20 Write an SQL query to find for each month and country, the number of transactions and their total amount, the number of approved transactions and their total amount.

Return the result table in any order.

The query result format is in the following example.

####Input: 
Transactions table:

| id   | country | state    | amount | trans_date |
|------|---------|----------|--------|------------|
| 121  | US      | approved | 1000   | 2018-12-18 |
| 122  | US      | declined | 2000   | 2018-12-19 |
| 123  | US      | approved | 2000   | 2019-01-01 |
| 124  | DE      | approved | 2000   | 2019-01-07 |

####Output: 
| month    | country | trans_count | approved_count | trans_total_amount | approved_total_amount |
|----------|---------|-------------|----------------|--------------------|-----------------------|
| 2018-12  | US      | 2           | 1              | 3000               | 1000                  |
| 2019-01  | US      | 1           | 1              | 2000               | 2000                  |
| 2019-01  | DE      | 1           | 1              | 2000               | 2000                  |

In [0]:
# imports 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, date_format, count, sum, when
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (121, "US", "approved", 1000, "2018-12-18"),
    (122, "US", "declined", 2000, "2018-12-19"),
    (123, "US", "approved", 2000, "2019-01-01"),
    (124, "DE", "approved", 2000, "2019-01-07")
]
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("country", StringType(), True),
    StructField("state", StringType(), True),
    StructField("amount", IntegerType(), True),
    StructField("trans_date", StringType(), True)  
])
df = spark.createDataFrame(data=data, schema=schema)
df = df.withColumn("trans_date", to_date(col("trans_date"), "yyyy-MM-dd"))

# solution
result_df = df.groupBy(date_format(col("trans_date"), "yyyy-MM").alias("month"), col("country")) \
    .agg(
        count("*").alias("trans_count"), 
        sum(col("amount")).alias("trans_total_amount"), 
        sum(when(col("state") == "approved", 1).otherwise(0)).alias("approved_count"), 
        sum(when(col("state") == "approved", col("amount")).otherwise(0)).alias("approved_total_amount")
    ) \
    .select(
        col("month"), 
        col("country"),
        col("trans_count"), 
        col("approved_count"), 
        col("trans_total_amount"),  
        col("approved_total_amount")
    )
display(result_df)

month,country,trans_count,approved_count,trans_total_amount,approved_total_amount
2018-12,US,2,1,3000,1000
2019-01,US,1,1,2000,2000
2019-01,DE,1,1,2000,2000


###Immediate Food Delivery II
If the customer's preferred delivery date is the same as the order date, then the order is called immediate; otherwise, it is called scheduled.

The first order of a customer is the order with the earliest order date that the customer made. It is guaranteed that a customer has precisely one first order.

####Q.21 Write a solution to find the percentage of immediate orders in the first orders of all customers, rounded to 2 decimal places.

The result format is in the following example.

####Input: 
Delivery table:

| delivery_id | customer_id | order_date | customer_pref_delivery_date |
|-------------|-------------|------------|-----------------------------|
| 1           | 1           | 2019-08-01 | 2019-08-02                  |
| 2           | 2           | 2019-08-02 | 2019-08-02                  |
| 3           | 1           | 2019-08-11 | 2019-08-12                  |
| 4           | 3           | 2019-08-24 | 2019-08-24                  |
| 5           | 3           | 2019-08-21 | 2019-08-22                  |
| 6           | 2           | 2019-08-11 | 2019-08-13                  |
| 7           | 4           | 2019-08-09 | 2019-08-09                  |

####Output: 
| immediate_percentage |
|----------------------|
| 50.00                |

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, row_number, format_number
from pyspark.sql.window import Window

# Spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# Input dataframe
data = [
    (1, 1, "2019-08-01", "2019-08-02"),
    (2, 2, "2019-08-02", "2019-08-02"),
    (3, 1, "2019-08-11", "2019-08-12"),
    (4, 3, "2019-08-24", "2019-08-24"),
    (5, 3, "2019-08-21", "2019-08-22"),
    (6, 2, "2019-08-11", "2019-08-13"),
    (7, 4, "2019-08-09", "2019-08-09")
]
schema = "delivery_id int, customer_id int, order_date string, customer_pref_delivery_date string"
df = spark.createDataFrame(data=data, schema=schema)
df = df.withColumn("order_date", to_date(col("order_date"), "yyyy-MM-dd"))

# Solution
window_spec = Window.partitionBy(col("customer_id")).orderBy(col("order_date"))
first_order_df = df.withColumn("order_rank", row_number().over(window_spec)) \
                   .filter(col("order_rank") == 1) \
                   .select(col("customer_id"), col("order_date"), col("customer_pref_delivery_date"))
first_order_count = first_order_df.count()
immediate_orders_count = first_order_df.filter(col("order_date") == col("customer_pref_delivery_date")).count()
data1 = [(first_order_count, immediate_orders_count)]
schema1 = "first_order_count int, immediate_orders_count int"
result_df = spark.createDataFrame(data=data1, schema=schema1)
result_df = result_df.withColumn("immediate_percentage",  \
                                 format_number((col("immediate_orders_count")/col("first_order_count"))*100, 2)) \
                                 .select(col("immediate_percentage"))
display(result_df)

immediate_percentage
50.00


###Game Play Analysis IV
####Q.22 Write a solution to report the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to count the number of players that logged in for at least two consecutive days starting from their first login date, then divide that number by the total number of players.

The result format is in the following example.

####Input: 
Activity table:

| player_id | device_id | event_date | games_played |
|-----------|-----------|------------|--------------|
| 1         | 2         | 2016-03-01 | 5            |
| 1         | 2         | 2016-03-02 | 6            |
| 2         | 3         | 2017-06-25 | 1            |
| 3         | 1         | 2016-03-02 | 0            |
| 3         | 4         | 2018-07-03 | 5            |

####Output: 
| fraction  |
------------|
| 0.33      |


In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, lead, lit
from pyspark.sql.window import Window

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (1, 2, "2016-03-01", 5),
    (1, 2, "2016-03-02", 6),
    (2, 3, "2017-06-25", 1),
    (3, 1, "2016-03-02", 0),
    (3, 4, "2018-07-03", 5)
]
schema = "player_id int, device_id int, event_date string, games_played int"
df = spark.createDataFrame(data=data, schema=schema)
df = df.withColumn("event_date", to_date(col("event_date"), "yyyy-MM-dd"))

# solution
window_spec = Window.partitionBy(col("player_id")).orderBy(col("event_date"))
consecutive_login_player_count = df.withColumn("following_event_date", lead(col("event_date"), 1).over(window_spec)) \
                       .filter(col("event_date")+1 == col("following_event_date")).distinct().count()
total_player_count = df.select(col("player_id")).distinct().count()
result_data = (consecutive_login_player_count/total_player_count)
result_df = spark.createDataFrame([(result_data,)], ["fraction"])
result_df = result_df.withColumn("fraction", round(col("fraction"), 2)).select(col("fraction"))
display(result_df)

fraction
0.33


###Number of Unique Subjects Taught by Each Teacher
####Q.23 Write a solution to calculate the number of unique subjects each teacher teaches in the university.

Return the result table in any order.

The result format is shown in the following example.

####Input: 
Teacher table:

| teacher_id | subject_id | dept_id |
|------------|------------|---------|
| 1          | 2          | 3       |
| 1          | 2          | 4       |
| 1          | 3          | 3       |
| 2          | 1          | 1       |
| 2          | 2          | 1       |
| 2          | 3          | 1       |
| 2          | 4          | 1       |

####Output:  
| teacher_id | cnt |
|------------|-----|
| 1          | 2   |
| 2          | 4   |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (1, 2, 3),
    (1, 2, 4),
    (1, 3, 3),
    (2, 1, 1),
    (2, 2, 1),
    (2, 3, 1),
    (2, 4, 1)
]
schema = "teacher_id int, subject_id int, dept_id int"
df = spark.createDataFrame(data=data, schema=schema)

# solution
result_df = df.groupBy(col("teacher_id")) \
              .agg(countDistinct(col("subject_id")).alias("cnt")) \
              .select(col("teacher_id"), col("cnt"))
display(result_df)

teacher_id,cnt
1,2
2,4


###User Activity for the Past 30 Days I
####Q.24 Write a solution to find the daily active user count for a period of 30 days ending 2019-07-27 inclusively. A user was active on someday if they made at least one activity on that day.

Return the result table in any order.

The result format is in the following example.

####Input: 
Activity table:

| user_id | session_id | activity_date | activity_type |
|---------|------------|---------------|---------------|
| 1       | 1          | 2019-07-20    | open_session  |
| 1       | 1          | 2019-07-20    | scroll_down   |
| 1       | 1          | 2019-07-20    | end_session   |
| 2       | 4          | 2019-07-20    | open_session  |
| 2       | 4          | 2019-07-21    | send_message  |
| 2       | 4          | 2019-07-21    | end_session   |
| 3       | 2          | 2019-07-21    | open_session  |
| 3       | 2          | 2019-07-21    | send_message  |
| 3       | 2          | 2019-07-21    | end_session   |
| 4       | 3          | 2019-06-25    | open_session  |
| 4       | 3          | 2019-06-25    | end_session   |

####Output:
| day        | active_users |
|------------|--------------|
| 2019-07-20 | 2            |
| 2019-07-21 | 2            |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, countDistinct, date_sub, lit

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (1, 1, "2019-07-20", "open_session"),
    (1, 1, "2019-07-20", "scroll_down"),
    (1, 1, "2019-07-20", "end_session"),
    (2, 4, "2019-07-20", "open_session"),
    (2, 4, "2019-07-21", "send_message"),
    (2, 4, "2019-07-21", "end_session"),
    (3, 2, "2019-07-21", "open_session"),
    (3, 2, "2019-07-21", "send_message"),
    (3, 2, "2019-07-21", "end_session"),
    (4, 3, "2019-06-25", "open_session"),
    (4, 3, "2019-06-25", "end_session")
]
schema = "user_id int, session_id int, activity_date string, activity_type string"
df = spark.createDataFrame(data=data, schema=schema)
df = df.withColumn("activity_date", to_date(col("activity_date"), "yyyy-MM-dd"))

# solution
start_date = '2019-07-27'
end_date = '2019-07-27'
result_df = df.filter((col("activity_date") >= date_sub(lit(start_date), 29)) & \
            (col("activity_date") <= lit(end_date)) & (col("activity_type").isNotNull())) \
            .groupBy(col("activity_date").alias("day")) \
            .agg(countDistinct(col("user_id")).alias("active_users")) \
            .select(col("day"), col("active_users")) \
            .orderBy(col("day"))
display(result_df)

day,active_users
2019-07-20,2
2019-07-21,2


###Product Sales Analysis III
####Q.25 Write a solution to select the product id, year, quantity, and price for the first year of every product sold.

Return the resulting table in any order.

The result format is in the following example.

####Input: 
Sales table:

| sale_id | product_id | year | quantity | price |
|---------|------------|------|----------|-------| 
| 1       | 100        | 2008 | 10       | 5000  |
| 2       | 100        | 2009 | 12       | 5000  |
| 7       | 200        | 2011 | 15       | 9000  |

Product table:

| product_id | product_name |
|------------|--------------|
| 100        | Nokia        |
| 200        | Apple        |
| 300        | Samsung      |

####Output: 
| product_id | first_year | quantity | price |
|------------|------------|----------|-------|
| 100        | 2008       | 10       | 5000  |
| 200        | 2011       | 15       | 9000  |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (1, 100, 2008, 10, 5000),
    (2, 100, 2009, 12, 5000),
    (7, 200, 2011, 15, 9000)
]
schema = "sale_id int, product_id int, year int, quantity int, price int"
df = spark.createDataFrame(data=data, schema=schema)

# solution
window_spec = Window.partitionBy(col("product_id")).orderBy(col("year"))
result_df = df.withColumn("year_rank", row_number().over(window_spec)) \
            .filter(col("year_rank") == 1) \
            .select(col("product_id"), col("year").alias("first_year"), col("quantity"), col("price"))
display(result_df)

product_id,first_year,quantity,price
100,2008,10,5000
200,2011,15,9000


###Classes More Than 5 Students
####Q.26 Write a solution to find all the classes that have at least five students.

Return the result table in any order.

The result format is in the following example.

####Input: 
Courses table:

| student | class    |
|---------|----------|
| A       | Math     |
| B       | English  |
| C       | Math     |
| D       | Biology  |
| E       | Math     |
| F       | Computer |
| G       | Math     |
| H       | Math     |
| I       | Math     |

####Output:
| class   |
|---------|
| Math    |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    ("A", "Math"),
    ("B", "English"),
    ("C", "Math"),
    ("D", "Biology"),
    ("E", "Math"),
    ("F", "Computer"),
    ("G", "Math"),
    ("H", "Math"),
    ("I", "Math")
]
schema = "student string, class string"
df = spark.createDataFrame(data=data, schema=schema)

# solution
result_df = df.groupBy(col("class")) \
            .agg(countDistinct(col("student")).alias("student_per_class")) \
            .filter(col("student_per_class") >= 5) \
            .select(col("class"))
display(result_df)

class
Math


###Find Followers Count
####Q.27 Write a solution that will, for each user, return the number of followers.

Return the result table ordered by user_id in ascending order.

The result format is in the following example.

####Input: 
Followers table:

| user_id | follower_id |
|---------|-------------|
| 0       | 1           |
| 1       | 0           |
| 2       | 0           |
| 2       | 1           |

####Output: 
| user_id | followers_count|
|---------|----------------|
| 0       | 1              |
| 1       | 1              |
| 2       | 2              |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (0, 1),
    (1, 0),
    (2, 0),
    (2, 1)
]
schema = "user_id int, follower_id int"
df = spark.createDataFrame(data=data, schema=schema)

# solution
result_df = df.groupBy(col("user_id")) \
            .agg(countDistinct(col("follower_id")).alias("followers_count")) \
            .select(col("user_id"), col("followers_count")) \
            .orderBy(col("user_id"))
display(result_df)

user_id,followers_count
0,1
1,1
2,2


###Biggest Single Number
####Q.28 A single number is a number that appeared only once in the MyNumbers table. Find the largest single number. If there is no single number, report null.

The result format is in the following example.

####Input: 
MyNumbers table:

| num |
|-----|
| 8   |
| 8   |
| 3   |
| 3   |
| 1   |
| 4   |
| 5   |
| 6   |

####Output:
| num |
|-----|
| 6   |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, max

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (8,),
    (8,),
    (3,),
    (3,),
    (1,),
    (4,),
    (5,),
    (6,)
]
schema = "num int"
df = spark.createDataFrame(data=data, schema=schema)

# solution
filter_df = df.groupBy(col("num")) \
            .agg(count(col("*")).alias("num_occurance")) \
            .filter(col("num_occurance") == 1) 
result_df = filter_df.select(col("num")).agg(max(col("num")).alias("num"))
display(result_df)

num
6


###Customers Who Bought All Products
####Q.29 Write a solution to report the customer ids from the Customer table that bought all the products in the Product table.

Return the result table in any order.

The result format is in the following example.

####Input: 
Customer table:

| customer_id | product_key |
|-------------|-------------|
| 1           | 5           |
| 2           | 6           |
| 3           | 5           |
| 3           | 6           |
| 1           | 6           |

Product table:

| product_key |
|-------------|
| 5           |
| 6           |

####Output:
| customer_id |
|-------------|
| 1           |
| 3           |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data1 = [
    (1, 5),
    (2, 6),
    (3, 5),
    (3, 6),
    (1, 6)
]
schema1 = "customer_id int, product_key int"
customer_df = spark.createDataFrame(data=data1, schema=schema1)
data2 = [
    (5,),
    (6,)
]
schema2 = "product_key int"
product_df = spark.createDataFrame(data=data2, schema=schema2)

# solution
unique_product_count = product_df.select(col("product_key")).count()
result_df = customer_df.groupBy(col("customer_id")) \
            .agg(countDistinct(col("product_key")).alias("purchase_count")) \
            .filter(col("purchase_count") == unique_product_count) \
            .select(col("customer_id")) \
            .orderBy(col("customer_id"))
display(result_df)

customer_id
1
3


###The Number of Employees Which Report to Each Employee
####Q.30 For this problem, we will consider a manager an employee who has at least 1 other employee reporting to them. Write a solution to report the ids and the names of all managers, the number of employees who report directly to them, and the average age of the reports rounded to the nearest integer.

Return the result table ordered by employee_id.

The result format is in the following example.

####Input: 
Employees table:

| employee_id | name    | reports_to | age |
|-------------|---------|------------|-----|
| 9           | Hercy   | null       | 43  |
| 6           | Alice   | 9          | 41  |
| 4           | Bob     | 9          | 36  |
| 2           | Winston | null       | 37  |

####Output: 
| employee_id | name  | reports_count | average_age |
|-------------|-------|---------------|-------------|
| 9           | Hercy | 2             | 39          |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, round

# Spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# Input DataFrame
data = [
    (9, "Hercy", None, 43),
    (6, "Alice", 9, 41),
    (4, "Bob", 9, 36),
    (2, "Winston", None, 37)
]
schema = "employee_id int, name string, reports_to int, age int"
df = spark.createDataFrame(data=data, schema=schema)
df = df.alias("df")

# solution
manager_reports_df = df.groupBy("reports_to") \
    .agg(
        count("*").alias("reports_count"),
        round(avg("age"), 0).alias("average_age")
    ) \
    .filter(col("reports_to").isNotNull()) \
    .alias("mgr")
result_df = manager_reports_df.join(df, col("mgr.reports_to") == col("df.employee_id")) \
    .select(
        col("df.employee_id"),
        col("df.name"),
        col("mgr.reports_count"),
        col("mgr.average_age")
    ) \
    .orderBy("df.employee_id")
display(result_df)

employee_id,name,reports_count,average_age
9,Hercy,2,39.0


###Primary Department for Each Employee
####Q.31 Employees can belong to multiple departments. When the employee joins other departments, they need to decide which department is their primary department. Note that when an employee belongs to only one department, their primary column is 'N'.Write a solution to report all the employees with their primary department. For employees who belong to one department, report their only department.

Return the result table in any order.

The result format is in the following example.

####Input: 
Employee table:

| employee_id | department_id | primary_flag |
|-------------|---------------|--------------|
| 1           | 1             | N            |
| 2           | 1             | Y            |
| 2           | 2             | N            |
| 3           | 3             | N            |
| 4           | 2             | N            |
| 4           | 3             | Y            |
| 4           | 4             | N            |

####Output: 
| employee_id | department_id |
|-------------|---------------|
| 1           | 1             |
| 2           | 1             |
| 3           | 3             |
| 4           | 3             |

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when

# Spark session
spark = SparkSession.builder.appName("PrimaryDepartment").getOrCreate()

# Input dataframe
data = [
    (1, 1, "N"),
    (2, 1, "Y"),
    (2, 2, "N"),
    (3, 3, "N"),
    (4, 2, "N"),
    (4, 3, "Y"),
    (4, 4, "N")
]
schema = "employee_id int, department_id int, primary_flag string"
df = spark.createDataFrame(data=data, schema=schema)

# solution
dept_count_df = df.groupBy("employee_id").agg(count("*").alias("dept_count"))
joined_df = df.join(dept_count_df, "employee_id")
result_df = joined_df.filter(
    (col("primary_flag") == "Y") |
    (col("dept_count") == 1)
).select(col("employee_id"), col("department_id"))
display(result_df)

employee_id,department_id
1,1
2,1
3,3
4,3


###Triangle Judgement
####Q.32 Report for every three line segments whether they can form a triangle.

Return the result table in any order.

The result format is in the following example.

####Input: 
Triangle table:

| x  | y  | z  |
|----|----|----|
| 13 | 15 | 30 |
| 10 | 20 | 15 |

####Output: 
| x  | y  | z  | triangle |
|----|----|----|----------|
| 13 | 15 | 30 | No       |
| 10 | 20 | 15 | Yes      |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (13, 15, 30),
    (10, 20, 15)
]
schema = "x int, y int, z int"
df = spark.createDataFrame(data=data, schema=schema)

# solution
result_df = df.withColumn(
        "triangle", 
        when((((col("x")+col("y")) > col("z")) & ((col("y")+col("z")) > col("x")) & ((col("z")+col("x")) > col("y"))), "Yes")
        .otherwise("No")
    ) \
    .select(col("x"), col("y"), col("z"), "triangle")
display(result_df)

x,y,z,triangle
13,15,30,No
10,20,15,Yes


###Consecutive Numbers
####Q.33 Find all numbers that appear at least three times consecutively.

Return the result table in any order.

The result format is in the following example.

####Input: 
Logs table:

| id | num |
|----|-----|
| 1  | 1   |
| 2  | 1   |
| 3  | 1   |
| 4  | 2   |
| 5  | 1   |
| 6  | 2   |
| 7  | 2   |

####Output: 
| ConsecutiveNums |
|-----------------|
| 1               |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag
from pyspark.sql.window import Window

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (1, 1),
    (2, 1),
    (3, 1),
    (4, 2),
    (5, 1),
    (6, 2),
    (7, 2)
]
schema = "id int, num int"
df = spark.createDataFrame(data=data, schema=schema)

# solution
window_spec = Window.orderBy("id")
df_with_lags = df.withColumn("prev_num1", lag("num", 1).over(window_spec)) \
                 .withColumn("prev_num2", lag("num", 2).over(window_spec))
consecutive_nums = df_with_lags.filter(
    (col("num") == col("prev_num1")) & (col("num") == col("prev_num2"))
).select(col("num").alias("ConsecutiveNums")).distinct()
display(consecutive_nums)

ConsecutiveNums
1


###Product Price at a Given Date
####Q.34 Write a solution to find the prices of all products on 2019-08-16. Assume the price of all products before any change is 10.

Return the result table in any order.

The result format is in the following example.

####Input: 
Products table:

| product_id | new_price | change_date |
|------------|-----------|-------------|
| 1          | 20        | 2019-08-14  |
| 2          | 50        | 2019-08-14  |
| 1          | 30        | 2019-08-15  |
| 1          | 35        | 2019-08-16  |
| 2          | 65        | 2019-08-17  |
| 3          | 20        | 2019-08-18  |

####Output: 
| product_id | price |
|------------|-------|
| 2          | 50    |
| 1          | 35    |
| 3          | 10    |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, max, lit, coalesce, desc
from pyspark.sql.window import Window

# spark session
spark = SparkSession.builder.appName("ProductPriceApp").getOrCreate()

# input dataframe
data = [
    (1, 20, '2019-08-14'),
    (2, 50, '2019-08-14'),
    (1, 30, '2019-08-15'),
    (1, 35, '2019-08-16'),
    (2, 65, '2019-08-17'),
    (3, 20, '2019-08-18')
]
schema = "product_id int, new_price int, change_date string"
products_df = spark.createDataFrame(data=data, schema=schema)

# solution
products_df = products_df.withColumn("change_date", to_date(col("change_date"), "yyyy-MM-dd"))
filtered_df = products_df.filter(col("change_date") <= "2019-08-16")
window_spec = Window.partitionBy(col("product_id")).orderBy(col("change_date").desc())
latest_price_df = filtered_df.withColumn("latest_price", max(col("new_price")).over(window_spec)) \
    .filter(col("latest_price").isNotNull()) \
    .groupBy(col("product_id")).agg(max(col("latest_price")).alias("price"))
all_products_df = products_df.select(col("product_id")).distinct()
result_df = all_products_df.join(latest_price_df, on="product_id", how="left_outer") \
    .withColumn("price", coalesce(col("price"), lit(10)))
result_df = result_df.select(col("product_id"), col("price")).orderBy(col("price").desc())
display(result_df)

product_id,price
2,50
1,35
3,10


###Last Person to Fit in the Bus
####Q.35 There is a queue of people waiting to board a bus. However, the bus has a weight limit of 1000 kilograms, so there may be some people who cannot board. Write a solution to find the person_name of the last person that can fit on the bus without exceeding the weight limit. The test cases are generated such that the first person does not exceed the weight limit.

The result format is in the following example.

####Input: 
Queue table:

| person_id | person_name | weight | turn |
|-----------|-------------|--------|------|
| 5         | Alice       | 250    | 1    |
| 4         | Bob         | 175    | 5    |
| 3         | Alex        | 350    | 2    |
| 6         | John Cena   | 400    | 3    |
| 1         | Winston     | 500    | 6    |
| 2         | Marie       | 200    | 4    |

####Output: 
| person_name |
|-------------|
| John Cena   |

In [0]:
# imports 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, desc
from pyspark.sql.window import Window

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (5, "Alice", 250, 1),
    (4, "Bob", 175, 5),
    (3, "Alex", 350, 2),
    (6, "John Cena", 400, 3),
    (1, "Winston", 500, 6),
    (2, "Marie", 200, 4)
]
schema = "person_id int, person_name string, weight int, turn int"
df = spark.createDataFrame(data=data, schema=schema)

# solution
window_spec = Window.orderBy(col("turn"))
result_df = df.withColumn("weight_per_turn", sum(col("weight")).over(window_spec)) \
    .filter(col("weight_per_turn") <= 1000) \
    .orderBy(desc(col("turn"))) \
    .select(col("person_name")).limit(1)
display(result_df)

person_name
John Cena


###Count Salary Categories
####Q.36 Write a solution to calculate the number of bank accounts for each salary category. The salary categories are:
- **"Low Salary":** All the salaries strictly less than $20000.
- **"Average Salary":** All the salaries in the inclusive range [$20000, $50000].
- **"High Salary":** All the salaries strictly greater than $50000.

The result table must contain all three categories. If there are no accounts in a category, return 0.

Return the result table in any order.

The result format is in the following example.

####Input: 
Accounts table:

| account_id | income |
|------------|--------|
| 3          | 108939 |
| 2          | 12747  |
| 8          | 87709  |
| 6          | 91796  |

####Output: 
| category       | accounts_count |
|----------------|----------------|
| Low Salary     | 1              |
| Average Salary | 0              |
| High Salary    | 3              |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (3, 108939),
    (2, 12747),
    (8, 87709),
    (6, 91796)
]
schema = "account_id int, income int"
df = spark.createDataFrame(data=data, schema=schema)

# solution
salary_category_df = df.withColumn("salary_category", 
                                   when(col("income") < 20000, "Low Salary") 
                                   .when((col("income") >= 20000) & (col("income") <= 50000), "Average Salary") 
                                   .otherwise("High Salary"))
category_count_df = salary_category_df.groupBy(col("salary_category")) \
            .agg(count(col("*")).alias("accounts_count")) \
            .select(col("salary_category"), col("accounts_count"))
categories = ["Low Salary", "Average Salary", "High Salary"]
category_df = spark.createDataFrame([(cat,) for cat in categories], ["salary_category"])
result_df = category_df.join(category_count_df, on="salary_category", how="left_outer").fillna(0)
display(result_df)

salary_category,accounts_count
Low Salary,1
Average Salary,0
High Salary,3


###Employees Whose Manager Left the Company
####Q.37 Find the IDs of the employees whose salary is strictly less than $30000 and whose manager left the company. When a manager leaves the company, their information is deleted from the Employees table, but the reports still have their manager_id set to the manager that left.

Return the result table ordered by employee_id.

The result format is in the following example.

####Input:  
Employees table:

| employee_id | name      | manager_id | salary |
|-------------|-----------|------------|--------|
| 3           | Mila      | 9          | 60301  |
| 12          | Antonella | null       | 31000  |
| 13          | Emery     | null       | 67084  |
| 1           | Kalel     | 11         | 21241  |
| 9           | Mikaela   | null       | 50937  |
| 11          | Joziah    | 6          | 28485  |

####Output:
| employee_id |
|-------------|
| 11          |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (3, "Mila", 9, 60301),
    (12, "Antonella", None, 31000),
    (13, "Emery", None, 67084),
    (1, "Kalel", 11, 21241),
    (9, "Mikaela", None, 50937),
    (11, "Joziah", 6, 28485)
]
schema = "employee_id int, name string, manager_id int, salary int"
df = spark.createDataFrame(data=data, schema=schema)

# solution
all_employee_df = df.select(col("employee_id")).distinct()
filter_df = df.filter((col("salary") < 30000) & (col("manager_id").isNotNull())) \
            .select(col("employee_id"), col("manager_id"))
manager_left_df = filter_df.select(col("manager_id")).subtract(all_employee_df).alias("manager_id")
result_df = manager_left_df.join(filter_df, on="manager_id", how="inner").select(col("employee_id"))
display(result_df)

employee_id
11


###Exchange Seats
####Q.38 Write a solution to swap the seat id of every two consecutive students. If the number of students is odd, the id of the last student is not swapped.

Return the result table ordered by id in ascending order.

The result format is in the following example.

####Input: 
Seat table:

| id | student |
|----|---------|
| 1  | Abbot   |
| 2  | Doris   |
| 3  | Emerson |
| 4  | Green   |
| 5  | Jeames  |

####Output: 
| id | student |
|----|---------|
| 1  | Doris   |
| 2  | Abbot   |
| 3  | Green   |
| 4  | Emerson |
| 5  | Jeames  |

In [0]:
# imports 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (1, "Abbot"),
    (2, "Doris"),
    (3, "Emerson"),
    (4, "Green"),
    (5, "Jeames")
]
schema = "id int, student string"
df = spark.createDataFrame(data=data, schema=schema)

# solution
student_count = df.count()
result_df = df.withColumn("new_id", 
                          when((col("id") % 2 == 0), col("id") - 1)
                          .when((col("id") % 2 != 0) & (col("id") < student_count), col("id") + 1)
                          .otherwise(col("id"))) \
                         .select(col("new_id").alias("id"), col("student")).orderBy(col("id"))
display(result_df)                 

id,student
1,Doris
2,Abbot
3,Green
4,Emerson
5,Jeames


###Movie Rating
####Q.39 Write a solution to:

- Find the name of the user who has rated the greatest number of movies. In case of a tie, return the lexicographically smaller user name.

- Find the movie name with the highest average rating in February 2020. In case of a tie, return the lexicographically smaller movie name.

The result format is in the following example.

####Input: 
Movies table:

| movie_id    |  title       |
|-------------|--------------|
| 1           | Avengers     |
| 2           | Frozen 2     |
| 3           | Joker        |

Users table:

| user_id     |  name        |
|-------------|--------------|
| 1           | Daniel       |
| 2           | Monica       |
| 3           | Maria        |
| 4           | James        |

MovieRating table:

| movie_id    | user_id      | rating       | created_at  |
|-------------|--------------|--------------|-------------|
| 1           | 1            | 3            | 2020-01-12  |
| 1           | 2            | 4            | 2020-02-11  |
| 1           | 3            | 2            | 2020-02-12  |
| 1           | 4            | 1            | 2020-01-01  |
| 2           | 1            | 5            | 2020-02-17  | 
| 2           | 2            | 2            | 2020-02-01  | 
| 2           | 3            | 2            | 2020-03-01  |
| 3           | 1            | 3            | 2020-02-22  | 
| 3           | 2            | 4            | 2020-02-25  | 

####Output: 
| results      |
|--------------|
| Daniel       |
| Frozen 2     |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, month, year, count, avg, lit
from pyspark.sql.window import Window

# spark session
spark = SparkSession.builder.appName("MovieRatingApp").getOrCreate()

# input dataframe
movies_data = [
    (1, 'Avengers'),
    (2, 'Frozen 2'),
    (3, 'Joker')
]
movies_schema = "movie_id int, title string"
movies_df = spark.createDataFrame(data=movies_data, schema=movies_schema)
users_data = [
    (1, 'Daniel'),
    (2, 'Monica'),
    (3, 'Maria'),
    (4, 'James')
]
users_schema = "user_id int, name string"
users_df = spark.createDataFrame(data=users_data, schema=users_schema)
ratings_data = [
    (1, 1, 3, '2020-01-12'),
    (1, 2, 4, '2020-02-11'),
    (1, 3, 2, '2020-02-12'),
    (1, 4, 1, '2020-01-01'),
    (2, 1, 5, '2020-02-17'),
    (2, 2, 2, '2020-02-01'),
    (2, 3, 2, '2020-03-01'),
    (3, 1, 3, '2020-02-22'),
    (3, 2, 4, '2020-02-25')
]
ratings_schema = "movie_id int, user_id int, rating int, created_at string"
ratings_df = spark.createDataFrame(data=ratings_data, schema=ratings_schema)

# solution
ratings_df = ratings_df.withColumn("created_at", to_date(col("created_at"), "yyyy-MM-dd"))
user_rating_count_df = ratings_df.groupBy(col("user_id")).agg(count("*").alias("rating_count"))
most_ratings_user_df = user_rating_count_df.join(users_df, on="user_id") \
                                           .orderBy(col("rating_count").desc(), col("name").asc()) \
                                           .limit(1)
february_ratings_df = ratings_df.filter((month(col("created_at")) == 2) & (year(col("created_at")) == 2020))
movie_avg_rating_df = february_ratings_df.groupBy(col("movie_id")).agg(avg(col("rating")).alias("avg_rating"))
highest_avg_rating_movie_df = movie_avg_rating_df.join(movies_df, on="movie_id") \
                                                 .orderBy(col("avg_rating").desc(), col("title").asc()) \
                                                 .limit(1)
result_df = most_ratings_user_df.select(col("name").alias("results")) \
    .union(highest_avg_rating_movie_df.select(col("title").alias("results")))
display(result_df)

results
Daniel
Frozen 2


###Restaurant Growth
####Q.40 You are the restaurant owner and you want to analyze a possible expansion (there will be at least one customer every day). 

Compute the moving average of how much the customer paid in a seven days window (i.e., current day + 6 days before). average_amount should be rounded to two decimal places.

Return the result table ordered by visited_on in ascending order.

The result format is in the following example.

####Input: 
Customer table:

| customer_id | name         | visited_on   | amount      |
|-------------|--------------|--------------|-------------|
| 1           | Jhon         | 2019-01-01   | 100         |
| 2           | Daniel       | 2019-01-02   | 110         |
| 3           | Jade         | 2019-01-03   | 120         |
| 4           | Khaled       | 2019-01-04   | 130         |
| 5           | Winston      | 2019-01-05   | 110         | 
| 6           | Elvis        | 2019-01-06   | 140         | 
| 7           | Anna         | 2019-01-07   | 150         |
| 8           | Maria        | 2019-01-08   | 80          |
| 9           | Jaze         | 2019-01-09   | 110         | 
| 1           | Jhon         | 2019-01-10   | 130         | 
| 3           | Jade         | 2019-01-10   | 150         | 

####Output: 
| visited_on   | amount       | average_amount |
|--------------|--------------|----------------|
| 2019-01-07   | 860          | 122.86         |
| 2019-01-08   | 840          | 120            |
| 2019-01-09   | 840          | 120            |
| 2019-01-10   | 1000         | 142.86         |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, sum, round, to_date, expr

# spark session
spark = SparkSession.builder.appName("RestaurantGrowth").getOrCreate()

# input dataframe
data = [
    (1, "Jhon", "2019-01-01", 100),
    (2, "Daniel", "2019-01-02", 110),
    (3, "Jade", "2019-01-03", 120),
    (4, "Khaled", "2019-01-04", 130),
    (5, "Winston", "2019-01-05", 110),
    (6, "Elvis", "2019-01-06", 140),
    (7, "Anna", "2019-01-07", 150),
    (8, "Maria", "2019-01-08", 80),
    (9, "Jaze", "2019-01-09", 110),
    (1, "Jhon", "2019-01-10", 130),
    (3, "Jade", "2019-01-10", 150)
]
schema = "customer_id int, name string, visited_on string, amount int"
df = spark.createDataFrame(data, schema)
df = df.withColumn("visited_on", to_date(col("visited_on"), "yyyy-MM-dd"))

# solution
daily_df = df.groupBy("visited_on").agg(sum("amount").alias("amount"))
window_spec = Window.orderBy("visited_on").rowsBetween(-6, 0)
result_df = daily_df.withColumn("rolling_amount", sum("amount").over(window_spec)) \
                    .withColumn("average_amount", round(col("rolling_amount") / 7, 2)) \
                    .select("visited_on", "rolling_amount", "average_amount") \
                    .orderBy("visited_on")
min_date = df.agg(expr("min(visited_on)")).collect()[0][0]
result_df = result_df.filter(col("visited_on") >= expr(f"DATE_ADD('{min_date}', 6)"))
result_df = result_df.withColumnRenamed("rolling_amount", "amount")
display(result_df)

visited_on,amount,average_amount
2019-01-07,860,122.86
2019-01-08,840,120.0
2019-01-09,840,120.0
2019-01-10,1000,142.86


###Friend Requests II: Who Has the Most Friends
####Q.41 Write a solution to find the people who have the most friends and the most friends number.

The test cases are generated so that only one person has the most friends.

The result format is in the following example.

####Input: 
RequestAccepted table:

| requester_id | accepter_id | accept_date |
|--------------|-------------|-------------|
| 1            | 2           | 2016/06/03  |
| 1            | 3           | 2016/06/08  |
| 2            | 3           | 2016/06/08  |
| 3            | 4           | 2016/06/09  |

####Output:
| id | num |
|----|-----|
| 3  | 3   |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (1, 2, '2016/06/03'),
    (1, 3, '2016/06/08'),
    (2, 3, '2016/06/08'),
    (3,	4, '2016/06/09')
]
schema = "requester_id int, accepter_id int, accept_date string"
df = spark.createDataFrame(data=data, schema=schema)

# solution
union_df = df.select(col("requester_id").alias("id")) \
     .unionAll(df.select(col("accepter_id").alias("id")))
result_df = union_df.groupBy(col("id")).agg(count(col("id")).alias("num")).orderBy(col("num").desc()).limit(1)
display(result_df)

id,num
3,3


###Investments in 2016
####Q.42 Write a solution to report the sum of all total investment values in 2016 tiv_2016, for all policyholders who:
- have the same tiv_2015 value as one or more other policyholders, and
- are not located in the same city as any other policyholder (i.e., the (lat, lon) attribute pairs must be unique).
Round tiv_2016 to two decimal places.

The result format is in the following example.

####Input: 
Insurance table:

| pid | tiv_2015 | tiv_2016 | lat | lon |
|-----|----------|----------|-----|-----|
| 1   | 10       | 5        | 10  | 10  |
| 2   | 20       | 20       | 20  | 20  |
| 3   | 10       | 30       | 20  | 20  |
| 4   | 10       | 40       | 40  | 40  |

####Output: 
| tiv_2016 |
|----------|
| 45.00    |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, format_number

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (1, 10, 5, 10, 10),
    (2, 20, 20, 20, 20),
    (3, 10, 30, 20, 20),
    (4, 10, 40, 40, 40)
]
schema = "pid int, tiv_2015 int, tiv_2016 int, lat int, lon int"
df = spark.createDataFrame(data=data, schema=schema)

# solution
duplicate_tiv_2015 = df.groupBy(col("tiv_2015")).count().filter(col("count") > 1).select(col("tiv_2015"))
unique_location = df.groupBy(col("lat"), col("lon")).count().filter(col("count") == 1).select(col("lat"), col("lon"))
filtered_df = df.join(duplicate_tiv_2015, on="tiv_2015").join(unique_location, on=["lat", "lon"])
result_df = filtered_df.agg(format_number(sum("tiv_2016"), 2).alias("tiv_2016"))
display(result_df)

tiv_2016
45.00


###Department Top Three Salaries
####Q.43 A company's executives are interested in seeing who earns the most money in each of the company's departments. A high earner in a department is an employee who has a salary in the top three unique salaries for that department.

Write a solution to find the employees who are high earners in each of the departments.

Return the result table in any order.

The result format is in the following example.

####Input: 
Employee table:

| id | name  | salary | departmentId |
|----|-------|--------|--------------|
| 1  | Joe   | 85000  | 1            |
| 2  | Henry | 80000  | 2            |
| 3  | Sam   | 60000  | 2            |
| 4  | Max   | 90000  | 1            |
| 5  | Janet | 69000  | 1            |
| 6  | Randy | 85000  | 1            |
| 7  | Will  | 70000  | 1            |

Department table:

| id | name  |
|----|-------|
| 1  | IT    |
| 2  | Sales |

####Output: 
| Department | Employee | Salary |
|------------|----------|--------|
| IT         | Max      | 90000  |
| IT         | Joe      | 85000  |
| IT         | Randy    | 85000  |
| IT         | Will     | 70000  |
| Sales      | Henry    | 80000  |
| Sales      | Sam      | 60000  |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, dense_rank
from pyspark.sql.window import Window

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data1 = [
    (1, "Joe", 85000, 1),
    (2, "Henry", 80000, 2),
    (3, "Sam", 60000, 2),
    (4, "Max", 90000, 1),
    (5, "Janet", 69000, 1),
    (6, "Randy", 85000, 1),
    (7, "Will", 70000, 1)
]
schema1 = "id int, employee_name string, salary int, departmentId int"
df1 = spark.createDataFrame(data=data1, schema=schema1)
data2 = [
    (1, "IT"),
    (2, "Sales")
]
schema2 = "id int, department_name string"
df2 = spark.createDataFrame(data=data2, schema=schema2)

# solution
joined_df = df1.join(df2, df1.departmentId == df2.id, "left_outer")
window_spec = Window.partitionBy("department_name").orderBy(col("salary").desc())
ranked_df = joined_df.withColumn("rank", dense_rank().over(window_spec))
result_df = ranked_df.filter(col("rank") <= 3) \
            .select(col("department_name").alias("Department"), col("employee_name").alias("Employee"), \
            col("salary").alias("Salary"))
display(result_df)

Department,Employee,Salary
IT,Max,90000
IT,Joe,85000
IT,Randy,85000
IT,Will,70000
Sales,Henry,80000
Sales,Sam,60000


###Fix Names in a Table
####Q.44 Write a solution to fix the names so that only the first character is uppercase and the rest are lowercase.

Return the result table ordered by user_id.

The result format is in the following example.

####Input:
Users table:

| user_id | name  |
|---------|-------|
| 1       | aLice |
| 2       | bOB   |

####Output: 
| user_id | name  |
|---------|-------|
| 1       | Alice |
| 2       | Bob   |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, substring, lower, upper

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (1, 'aLice'),
    (2, 'bOB')
]
schema = "user_id int, name string"
df = spark.createDataFrame(data=data, schema=schema)

# solution
result_df = df.withColumn("name", concat(upper(substring(col("name"), 1, 1)), lower(substring(col("name"), 2, 100)))).orderBy(col("name"))
display(result_df)

user_id,name
1,Alice
2,Bob


###Patients With a Condition
####Q.45 Write a solution to find the patient_id, patient_name, and conditions of the patients who have Type I Diabetes. Type I Diabetes always starts with DIAB1 prefix.

Return the result table in any order.

The result format is in the following example.

####Input: 
Patients table:

| patient_id | patient_name | conditions   |
|------------|--------------|--------------|
| 1          | Daniel       | YFEV COUGH   |
| 2          | Alice        |              |
| 3          | Bob          | DIAB100 MYOP |
| 4          | George       | ACNE DIAB100 |
| 5          | Alain        | DIAB201      |

####Output: 
| patient_id | patient_name | conditions   |
|------------|--------------|--------------|
| 3          | Bob          | DIAB100 MYOP |
| 4          | George       | ACNE DIAB100 | 

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (1, 'Daniel', 'YFEV COUGH'),
    (2, 'Alice', None),	
    (3,	'Bob', 'DIAB100 MYOP'),
    (4, 'George', 'ACNE DIAB100'),
    (5,	'Alain', 'DIAB201')
]
schema = "patient_id int, patient_name string, conditions string"
df = spark.createDataFrame(data=data, schema=schema)

# solution
result_df = df.filter(col("conditions").like("%DIAB1%")).select(col("patient_id"), col("patient_name"), col("conditions"))
display(result_df)

patient_id,patient_name,conditions
3,Bob,DIAB100 MYOP
4,George,ACNE DIAB100


###Delete Duplicate Emails
####Q.46 Write a solution to delete all duplicate emails, keeping only one unique email with the smallest id. 

For SQL users, please note that you are supposed to write a DELETE statement and not a SELECT one.

For Pandas users, please note that you are supposed to modify Person in place.

After running your script, the answer shown is the Person table. The driver will first compile and run your piece of code and then show the Person table. The final order of the Person table does not matter.

The result format is in the following example.

####Input: 
Person table:

| id | email            |
|----|------------------|
| 1  | john@example.com |
| 2  | bob@example.com  |
| 3  | john@example.com |

####Output: 
| id | email            |
|----|------------------|
| 1  | john@example.com |
| 2  | bob@example.com  |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (1, 'john@example.com'),
    (2, 'bob@example.com'),
    (3, 'john@example.com')
]
schema = "id int, email string"
df = spark.createDataFrame(data=data, schema=schema)

# solution
window_spec = Window.orderBy(col("id")).partitionBy(col("email"))
result_df = df.withColumn("ranking", row_number().over(window_spec)).filter(col("ranking") == 1) \
        .select(col("id"), col("email")).orderBy(col("id"))
display(result_df)

id,email
1,john@example.com
2,bob@example.com


###Second Highest Salary
####Q.47 Write a solution to find the second highest salary from the Employee table. If there is no second highest salary, return null (return None in Pandas).

The result format is in the following example.

####Input: 
Employee table:

| id | salary |
|----|--------|
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |

####Output: 
| SecondHighestSalary |
|---------------------|
| 200                 |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import col, desc, dense_rank, when, max
from pyspark.sql.window import Window

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (1, 100),
    (2, 200),
    (3, 300)
]
schema = "id int, salary int"
df = spark.createDataFrame(data=data, schema=schema)

# solution
window_spec = Window.orderBy(col("salary").desc())
rank_df = df.withColumn("dense_ranking", dense_rank().over(window_spec))
max_rank = rank_df.select(max(col("dense_ranking"))).collect()[0][0]
result_df = rank_df.withColumn(
    "SecondHighestSalary",
    when(col("dense_ranking") >= 2, col("salary"))
).filter(col("dense_ranking") == 2).select(col("SecondHighestSalary")).distinct()
if max_rank < 2:
    result_df = spark.createDataFrame([(None,)], StructType([StructField("SecondHighestSalary", IntegerType(), True)]))
display(result_df)

SecondHighestSalary
200


###Group Sold Products By The Date
####Q.48 Write a solution to find for each date the number of different products sold and their names.

The sold products names for each date should be sorted lexicographically.

Return the result table ordered by sell_date.

The result format is in the following example.

####Input: 
Activities table:

| sell_date  | product    |
|------------|------------|
| 2020-05-30 | Headphone  |
| 2020-06-01 | Pencil     |
| 2020-06-02 | Mask       |
| 2020-05-30 | Basketball |
| 2020-06-01 | Bible      |
| 2020-06-02 | Mask       |
| 2020-05-30 | T-Shirt    |

####Output: 
| sell_date  | num_sold | products                     |
|------------|----------|------------------------------|
| 2020-05-30 | 3        | Basketball,Headphone,T-shirt |
| 2020-06-01 | 2        | Bible,Pencil                 |
| 2020-06-02 | 1        | Mask                         |

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, countDistinct, collect_set, sort_array, array_join

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    ("2020-05-30", "Headphone"),
    ("2020-06-01", "Pencil"),
    ("2020-06-02", "Mask"),
    ("2020-05-30", "Basketball"),
    ("2020-06-01", "Bible"),
    ("2020-06-02", "Mask"),
    ("2020-05-30", "T-Shirt")
]
schema = "sell_date string, product string"
df = spark.createDataFrame(data=data, schema=schema)
df = df.withColumn("sell_date", to_date(col("sell_date"), "yyyy-MM-dd"))

# solution
result_df = df.groupBy(col("sell_date")) \
    .agg(
        countDistinct(col("product")).alias("num_sold"),
        array_join(sort_array(collect_set(col("product"))), delimiter=",").alias("products")
    ) \
    .orderBy(col("sell_date"))

display(result_df)

sell_date,num_sold,products
2020-05-30,3,"Basketball,Headphone,T-Shirt"
2020-06-01,2,"Bible,Pencil"
2020-06-02,1,Mask


###List the Products Ordered in a Period
####Q.49 Write a solution to get the names of products that have at least 100 units ordered in February 2020 and their amount.

Return the result table in any order.

The result format is in the following example.

####Input: 
Products table:

| product_id  | product_name          | product_category |
|-------------|-----------------------|------------------|
| 1           | Leetcode Solutions    | Book             |
| 2           | Jewels of Stringology | Book             |
| 3           | HP                    | Laptop           |
| 4           | Lenovo                | Laptop           |
| 5           | Leetcode Kit          | T-shirt          |

Orders table:

| product_id   | order_date   | unit     |
|--------------|--------------|----------|
| 1            | 2020-02-05   | 60       |
| 1            | 2020-02-10   | 70       |
| 2            | 2020-01-18   | 30       |
| 2            | 2020-02-11   | 80       |
| 3            | 2020-02-17   | 2        |
| 3            | 2020-02-24   | 3        |
| 4            | 2020-03-01   | 20       |
| 4            | 2020-03-04   | 30       |
| 4            | 2020-03-04   | 60       |
| 5            | 2020-02-25   | 50       |
| 5            | 2020-02-27   | 50       |
| 5            | 2020-03-01   | 50       |

####Output: 
| product_name       | unit    |
|--------------------|---------|
| Leetcode Solutions | 130     |
| Leetcode Kit       | 100     |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, year, month, sum

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data1 = [
    (1, 'Leetcode Solutions', 'Book'),
    (2, 'Jewels of Stringology', 'Book'),
    (3, 'HP', 'Laptop'),
    (4, 'Lenovo', 'Laptop'),
    (5, 'Leetcode Kit', 'T-shirt')
]
schema1 = "product_id int, product_name string, product_category string"
df1 = spark.createDataFrame(data=data1, schema=schema1)
data2 = [
    (1, '2020-02-05', 60),
    (1, '2020-02-10', 70),
    (2, '2020-01-18', 30),
    (2, '2020-02-11', 80),
    (3,	'2020-02-17', 2),
    (3, '2020-02-24', 3),
    (4,	'2020-03-01', 20),
    (4, '2020-03-04', 30),
    (4,	'2020-03-04', 60),
    (5,	'2020-02-25', 50),
    (5,	'2020-02-27', 50),
    (5,	'2020-03-01', 50)
]
schema2 = "product_id int, order_date string, unit int"
df2 = spark.createDataFrame(data=data2, schema=schema2)
df2 = df2.withColumn("order_date", to_date(col("order_date")))

# solution
join_df = df2.join(df1, on="product_id", how="inner")
group_df = join_df.filter((year(col("order_date")) == 2020) & (month(col("order_date")) == 2)) \
    .groupBy(col("product_id"), col("product_name")) \
    .agg(sum(col("unit")).alias("unit")) \
    .select(col("product_name"), col("unit"))
result_df = group_df.filter(col("unit") >= 100)
display(result_df)

product_name,unit
Leetcode Solutions,130
Leetcode Kit,100


###Find Users With Valid E-Mails
###Q.50 Write a solution to find the users who have valid emails.

A valid e-mail has a prefix name and a domain where:

- The prefix name is a string that may contain letters (upper or lower case), digits, underscore '_', period '.', and/or dash '-'. The prefix name must start with a letter.
- The domain is '@leetcode.com'.
Return the result table in any order.

The result format is in the following example.

####Input: 
Users table:

| user_id | name      | mail                    |
|---------|-----------|-------------------------|
| 1       | Winston   | winston@leetcode.com    |
| 2       | Jonathan  | jonathanisgreat         |
| 3       | Annabelle | bella-@leetcode.com     |
| 4       | Sally     | sally.come@leetcode.com |
| 5       | Marwan    | quarz#2020@leetcode.com |
| 6       | David     | david69@gmail.com       |
| 7       | Shapiro   | .shapo@leetcode.com     |

####Output: 
| user_id | name      | mail                    |
|---------|-----------|-------------------------|
| 1       | Winston   | winston@leetcode.com    |
| 3       | Annabelle | bella-@leetcode.com     |
| 4       | Sally     | sally.come@leetcode.com |

In [0]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# spark session
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

# input dataframe
data = [
    (1, 'Winston', 'winston@leetcode.com'), 
    (2, 'Jonathan', 'jonathanisgreat'),
    (3,	'Annabelle', 'bella-@leetcode.com'),
    (4,	'Sally', 'sally.come@leetcode.com'),
    (5,	'Marwan', 'quarz#2020@leetcode.com'),
    (6,	'David', 'david69@gmail.com'),
    (7,	'Shapiro', '.shapo@leetcode.com')
]
schema = "user_id int, name string, mail string"
df = spark.createDataFrame(data=data, schema=schema)

# solution
pattern = r'^[A-Za-z][A-Za-z0-9._-]*@leetcode\.com$'
result_df = df.filter(col("mail").rlike(pattern)).orderBy(col("user_id"))
display(result_df)

user_id,name,mail
1,Winston,winston@leetcode.com
3,Annabelle,bella-@leetcode.com
4,Sally,sally.come@leetcode.com


###